In [1]:
import os
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt

In [2]:
kingfa_datasets_path = "./kingfadatasets/"

In [3]:
def wpt1D(data, wavelet='db1', mode='symmetric', maxlevel=4):
    wp = pywt.WaveletPacket(data=data[1750:2261], wavelet=wavelet, mode=mode, maxlevel=maxlevel)
    paths = [node.path for node in wp.get_level(4)]  # 列出所有树节点路径
    df_wpt = pd.DataFrame()  # 合并所有四级子节点
    for a in paths:
        df_wpt[a] = wp[a].data
    return df_wpt.values

In [4]:
X = np.array(pd.read_csv(kingfa_datasets_path+"data.csv", index_col=0))

In [5]:
X = np.apply_along_axis(wpt1D, 1, X)

In [6]:
y = np.array(pd.read_csv(kingfa_datasets_path+"target.csv", index_col=0))

In [7]:
print(X.shape)
print(y.shape)

(3000, 32, 16)
(3000, 1)


In [31]:
X = X.reshape(-1, 32, 16, 1)
y = keras.utils.to_categorical(y, 3)

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [23]:
def create_model(optimizer='sgd', loss='squared_hinge'):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(7, 5), input_shape=(32, 16, 1)))
    model.add(Activation('sigmoid'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Conv2D(filters=32, kernel_size=(5, 3), padding='same'))
    model.add(Activation('sigmoid'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('sigmoid'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [12]:
seed = 1024
np.random.seed(seed)

In [24]:
model = KerasClassifier(build_fn=create_model)

In [32]:
optimizers = ['sgd', 'rmsprop', 'adam']
loss = ['mse', 'squared_hinge', 'categorical_hinge', 'categorical_crossentropy']
epochs = np.array([50, 100, 150, 200, 250, 300, 500])
batches = np.array([8, 16, 32, 64])

In [27]:
param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, loss=loss)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [33]:
grid_result = grid.fit(X, y)

Epoch 1/1
2000/2000 [==============================] - 0s 196us/step
Epoch 1/1
2000/2000 [==============================] - 0s 190us/step
Epoch 1/1
2000/2000 [==============================] - 0s 200us/step
Epoch 1/1
2000/2000 [==============================] - 0s 204us/step
Epoch 1/1
2000/2000 [==============================] - 0s 213us/step
Epoch 1/1
 720/2000 [=========>....................] - ETA: 0s

KeyboardInterrupt: 

In [34]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

NameError: name 'grid_result' is not defined